In [1]:
import cv2  
import numpy as np 
import matplotlib.pyplot as plt

# File Path

In [2]:
# img_path = '../raw_data/eye_pics/lena.jpg'
img_path = '../raw_data/eye_pics/blink_pn003.png'
img_path = '../raw_data/eyeMovement_test/em06/00897.png'
fimg = cv2.imread(img_path)  #Canny只能处理灰度图，所以将读取的图像转成灰度图
gimg = cv2.imread(img_path, 0)  #Canny只能处理灰度图，所以将读取的图像转成灰度图

# Canny Edge Detection

In [3]:
# fimg = cv2.imread(img_path)  #Canny只能处理灰度图，所以将读取的图像转成灰度图
# gimg = cv2.imread(img_path, 0)  #Canny只能处理灰度图，所以将读取的图像转成灰度图
# cv2.imshow('Original pic', fimg) 
# cv2.imshow('Gray pic', gimg)

# img = cv2.GaussianBlur(gimg,(3,3),0) #高斯平滑处理原图像降噪 
# canny = cv2.Canny(img, 50, 150)     #apertureSize默认为3
# # canny = cv2.Canny(canny, 50, 150)     #apertureSize默认为3

# cv2.imshow('Canny pic', canny)  
# cv2.waitKey(0)  
# cv2.destroyAllWindows()

# Hough Transformation

## 参数使用说明
|参数名称|说明|
|-|-|
|image|输入图像，需要灰度图|
|method|cv2.HOUGH_GRADIENT 也就是霍夫圆检测，梯度法|
|dp| 计数器的分辨率图像像素分辨率与参数空间分辨率的比值（官方文档上写的是图像分辨率与累加器分辨率的比值，它把参数空间认为是一个累加器，毕竟里面存储的都是经过的像素点的数量），dp=1，则参数空间与图像像素空间（分辨率）一样大，dp=2，参数空间的分辨率只有像素空间的一半大|
|minDist|圆心之间最小距离，如果距离太小，会产生很多相交的圆，如果距离太大，则会漏掉正确的圆|
|param1|canny检测的双阈值中的高阈值，低阈值是它的一半|
|param2|最小投票数（基于圆心的投票数）|
|minRadius|需要检测院的最小半径|
|maxRadius|需要检测院的最大半径|

In [4]:
# img = canny.copy()
# # img = gimg.copy()
# #跳过 canny 直接送入 Hought Transform 会有问题

# # img = dst.copy()
# # cv2.imshow("Gray pic", img)  
# # img_path = '../raw_data/eye_pics/ts_eye.png'
# # img = cv2.imread(img_path)
# # img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#灰度图像 
# # plt.subplot(121)
# # plt.imshow(img,'gray')
# #hough transform
# circles1 = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=5, minRadius=20, maxRadius=50)
# circles = circles1[0,:,:]#提取为二维
# circles = np.uint16(np.around(circles))#四舍五入，取整
# ht_img = fimg.copy()
# for i in circles[:]: 
#     cv2.circle(ht_img,(i[0],i[1]),i[2],(255,0,255),1)#画圆
#     cv2.circle(ht_img,(i[0],i[1]),2,(0,0,255),3)#画圆心

# # plt.subplot(122),plt.imshow(img)
# # plt.xticks([]),plt.yticks([])
# cv2.imshow("Hough_Transform pic", ht_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [5]:
# # x, y, radius
# for i in circles[:]: 
#     print(i)


In [6]:
import cv2  
import numpy as np  
from matplotlib import pyplot as plt  
from PIL import Image
  
img=cv2.imread(img_path)  #读取图片，装换为可运算的数组
GrayImage=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)   #将BGR图转为灰度图
ret,thresh=cv2.threshold(GrayImage,2,255,cv2.THRESH_BINARY)  #将图片进行二值化（130,255）之间的点均变为255（背景）
# print(thresh1[0,0])#250  输出[0,0]这个点的像素值  				#返回值ret为阈值
# print(ret)#130

(h,w) = thresh.shape #返回高和宽
half_w = w//2
# print(h,w)#s输出高和宽

# 垂直投影

In [7]:
a = [0 for z in range(0, w)] 
# print(a) #a = [0,0,0,0,0,0,0,0,0,0,...,0,0]初始化一个长度为w的数组，用于记录每一列的黑点个数  

thresh1 = thresh.copy() 
#记录每一列的波峰
for j in range(0,w): #遍历一列 
    for i in range(0,h):  #遍历一行
        if  thresh1[i,j] == 255:  #如果为白点
            a[j]+=1  		#该列的计数器加一计数
            thresh1[i,j]=0  #记录完后将其变为黑色 
            
    
for j  in range(0,w):  #遍历每一列
    for i in range(a[j]):  #从最顶部的点开始向最底部涂白
        thresh1[i,j] = 255   #涂白
 
#此时的thresh1便是一张图像向垂直方向上投影的直方图
#如果要分割字符的话，其实并不需要把这张图给画出来，只需要的到a=[]即可得到想要的信息
# cv2.waitKey(0)  
# cv2.destroyAllWindows()  

# 水平投影

In [8]:
thresh2 = thresh.copy() 
bl = [0 for z in range(0, h)]
br = [0 for z in range(0, h)] 
# print(a) 
 
for j in range(0,h):  
    for i in range(0,w):  
        if  thresh2[j,i]==255:
            if i < half_w:
                bl[j]+=1
            else:
                br[j]+=1
            thresh2[j,i]=0

for j  in range(0,h):  
    for i in range(0,bl[j]):
        thresh2[j,i] = 255
    for i in range(0,br[j]):
        thresh2[j,w - 1 - i] = 255

# plt.imshow(thresh1,cmap=plt.gray())
# plt.show()

# 可视化

In [9]:
# cv2.imshow('thresh',thresh) 
# cv2.imshow('thresh1',thresh1)  
# cv2.imshow('thresh2',thresh2) 
# cv2.waitKey(0)  
# cv2.destroyAllWindows()  

In [10]:
print("a max pos", a.index(max(a)))
print("bl max pos", bl.index(max(bl)))
print("br max pos", br.index(max(br)))

a max pos 68
bl max pos 131
br max pos 125


In [11]:
hl_max = bl.index(max(bl))
hr_max = br.index(max(br))
al = a[:w//2]
ar = a[w//2:]
vl_max = a.index(max(al))
vr_max = a.index(max(ar), w//2,)

In [12]:
trh = 1
# lu
cnt = hl_max
for i in bl[hl_max::-1]:
    if i < trh:
        lu = cnt
        break
    cnt = cnt - 1
    
# ld
cnt = hl_max
for i in bl[hl_max::]:
    if i < trh:
        ld = cnt
        break
    cnt = cnt + 1

# ru
cnt = hr_max
for i in br[hr_max::-1]:
    if i < trh:
        ru = cnt
        break
    cnt = cnt - 1

# rd
cnt = hr_max
for i in br[hr_max::]:
    if i < trh:
        rd = cnt
        break
    cnt = cnt + 1

# ll
cnt = vl_max
for i in a[vl_max::-1]:
    if i < trh:
        ll = cnt
        break
    cnt = cnt - 1
    
# lr
cnt = vl_max
for i in a[vl_max::]:
    if i < trh:
        lr = cnt
        break
    cnt = cnt + 1

# rl
cnt = vr_max
for i in a[vr_max::-1]:
    if i < trh:
        rl = cnt
        break
    cnt = cnt - 1

# rr
cnt = vr_max
for i in a[vr_max::]:
    if i < trh:
        rr = cnt
        break
    cnt = cnt + 1

In [13]:
img = fimg.copy()
cv2.rectangle(img, (ll,lu), (lr,ld), (0,0,255), 1)
cv2.rectangle(img, (rl,ru), (rr,rd), (0,255,255), 1)
# cv2.imshow('img',img) 
# cv2.waitKey(0)  
# cv2.destroyAllWindows()  

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [14]:
cv2.imshow('thresh',thresh) 
cv2.imshow('thresh1',thresh1)  
cv2.imshow('thresh2',thresh2) 
cv2.imshow('img',img) 
cv2.waitKey(0)  
cv2.destroyAllWindows()  

In [15]:
# #读入图像并转化为float类型，用于传递给harris函数
# # filename = 'Dirk.jpg'
 
# # img = cv2.imread(filename)
 
# gray_img = gimg.copy()
# # cv2.imshow('show1', gray_img)
# # cv2.waitKey(0)  
# # cv2.destroyAllWindows()
# gray_img = np.float32(gray_img)
# gray_img = cv2.GaussianBlur(gray_img,(3,3),0) #高斯平滑处理原图像降噪 
# #对图像执行harris
# dst = cv2.cornerHarris(gray_img, 2, 3, 0.04)
 
# #腐蚀harris结果
# dst = cv2.dilate(dst, None)
 
# # 设置阈值
# thres = 0.01*dst.max()

# fimg_ = fimg.copy() 
 
# fimg_[dst > thres] = [0,0,255]
# # img[dst>0.01*dst.max()]=[0,0,255]

# # dst = cv2.cornerHarris(gray_img,2,3,0.04)
# # dst = cv2.dilate(dst,None)
# # ret, dst = cv2.threshold(dst,0.01*dst.max(),255,3)
# # dst = np.uint8(dst)
 
# cv2.imshow('show', fimg_)
# cv2.waitKey(0)  
# cv2.destroyAllWindows()